<a href="https://colab.research.google.com/github/jinoh5/disentangled/blob/main/find_nondisentangled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>